In [10]:
from tensorflow import keras
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from google.colab import drive
from google.colab import files

Подключаемся к Google Drive

In [11]:
drive.mount('/content/drive')
# путь к директории с датасетом "flowers"
data_dir = '/content/drive/MyDrive/flowers'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Создаём генераторы для обучения и проверки

In [12]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(data_dir,target_size=(224,224), 
                                                    batch_size=32, class_mode='categorical', subset = 'training')
val_generator = train_datagen.flow_from_directory(data_dir, target_size=(224,224),
                                                  batch_size=32, class_mode='categorical', subset = 'validation')

Found 3474 images belonging to 5 classes.
Found 864 images belonging to 5 classes.


In [13]:
# загружаем предварительно обученную машину
model = keras.applications.VGG16(include_top=True, weights='imagenet')

In [20]:
# заменяем последний слой на новый с 5 выходами
last_layer = model.get_layer('fc2').output
new_output = keras.layers.Dense(5, activation = 'softmax', name='new_output')(last_layer)
model = keras.Model(inputs=model.input, outputs=new_output)

In [21]:
# замораживаем все слои, кроме нового слоя
for layer in model.layers[:-1]:
  layer.trainable = False

In [23]:
# компилируем модель и запускаем обучение 
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(train_generator, epochs=10, validation_data = val_generator)


Epoch 1/10
109/109 [==============================] - 708s 6s/step - loss: 1.1354 - accuracy: 0.5478 - val_loss: 0.8931 - val_accuracy: 0.6956
Epoch 2/10
109/109 [==============================] - 22s 200ms/step - loss: 0.8530 - accuracy: 0.6842 - val_loss: 0.8157 - val_accuracy: 0.6725
Epoch 3/10
109/109 [==============================] - 22s 201ms/step - loss: 0.7495 - accuracy: 0.7343 - val_loss: 0.7128 - val_accuracy: 0.7558
Epoch 4/10
109/109 [==============================] - 23s 208ms/step - loss: 0.7087 - accuracy: 0.7453 - val_loss: 0.8027 - val_accuracy: 0.6875
Epoch 5/10
109/109 [==============================] - 22s 199ms/step - loss: 0.6995 - accuracy: 0.7398 - val_loss: 0.7779 - val_accuracy: 0.7222
Epoch 6/10
109/109 [==============================] - 23s 206ms/step - loss: 0.6539 - accuracy: 0.7562 - val_loss: 0.6655 - val_accuracy: 0.7546
Epoch 7/10
109/109 [==============================] - 23s 208ms/step - loss: 0.6344 - accuracy: 0.7683 - val_loss: 0.7552 - val_accu

In [25]:
# оцениваем качество модели на тестовых данных
test_generator = train_datagen.flow_from_directory(data_dir, target_size=(224,224), 
                                                   batch_size=32, class_mode='categorical')
test_loss, test_acc = model.evaluate(test_generator)
print('Test accuracy: ', test_acc)

Found 4338 images belonging to 5 classes.
136/136 [==============================] - 23s 163ms/step - loss: 0.5591 - accuracy: 0.7978
Test accuracy:  0.7978330850601196


In [28]:
# загрузка изображения
uploaded = files.upload()
img_path = list(uploaded.keys())[0]

Saving 5547758_eea9edfd54_n.jpg to 5547758_eea9edfd54_n (1).jpg


In [29]:
# загружаем изображение и подготоваливаем его для передачи
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x /255.0

In [30]:
# получаем предсказание модели
preds = model.predict(x)
class_idx = np.argmax(preds)
class_name = list(train_generator.class_indices.keys())[class_idx]
# выводим результаты
print(f'Класс: {class_name}, вероятность: {preds[0][class_idx]}')

1/1 [==============================] - 1s 963ms/step
Класс: daisy, вероятность: 0.9466358423233032
